In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
use database la_permit_data;
use schema public;
select distinct permit_category from permit_records;

In [ ]:
select pcis_permit_num, status, status_date, year(status_date) as status_year,
permit_type, permit_sub_type, initiating_office, issue_date, year(issue_date) as issue_year, concat(address_start,' ',coalesce(street_direction,''), street_name,' ',street_suffix,' ',coalesce(suffix_direction,'')) as address,zip_code,work_description,ai_description,valuation,license_num,
contractor_business_name,license_type,census_tract,latitude_longitude
from la_permit_data.public.permit_records;

In [ ]:
import pandas as pd
permits=sql_cell1.to_pandas()
permits.head()

In [ ]:
permits[['LONGITUDE','LATITUDE']]=permits['LATITUDE_LONGITUDE']\
                                .str.extract(r'\(([^\s]+)\s([^\s]+)\)')
permits['LONGITUDE']=pd.to_numeric(permits['LONGITUDE'])
permits['LATITUDE']=pd.to_numeric(permits['LATITUDE'])
permits[['LONGITUDE','LATITUDE','LATITUDE_LONGITUDE']].info()

In [ ]:
import streamlit as st
st.map(permits[['LONGITUDE', 'LATITUDE']].dropna().head(100))

In [ ]:
permits['ISSUE_DATE'] = pd.to_datetime(permits['ISSUE_DATE'])
permits['VALUATION'] = pd.to_numeric(permits['VALUATION'].str.replace(r'[,$]', '',regex=True), errors='coerce')
permits.info()
permits.head()

In [ ]:
permits['ISSUE_DATE'].max()

In [ ]:
permits.groupby('ISSUE_YEAR')['PCIS_PERMIT_NUM'].nunique().iloc[:-1].plot(figsize=(6,2))

In [ ]:
select * from la_permit_data.public.permit_records limit 5;

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Create dataframe for renovations only
ren = permits[permits['PERMIT_TYPE'] == 'Bldg-Alter/Repair']

# Group by and count
top10 = ren.groupby('ZIP_CODE')['PCIS_PERMIT_NUM'].count().sort_values(ascending=False).head(10)

# Graph the top 10 neighborhoods
top10.plot(kind='bar', title='Top 10 Zip Codes for Renovations by Volume')

In [ ]:
# Create dataframe for renovations only
trends = permits[permits['PERMIT_TYPE'] == 'Bldg-Alter/Repair']

# Group by the month
monthly_trends = trends.groupby(pd.Grouper(key='ISSUE_DATE', freq='ME'))['PCIS_PERMIT_NUM'].count()

# Graph the line plot
monthly_trends.plot(title = 'Market Momentum: Monthly Renovation (2013-2023)')

In [ ]:
# Group by zipcode and find average valuation for top 10
top_10 = trends.groupby('ZIP_CODE')['VALUATION'].mean().sort_values(ascending=False).head(10)
top_10.plot(kind='bar', title='Top 10 Zip Codes Average Valuation')

In [ ]:
# Filter for renovations under $100k
ren_budget = ren[ren['VALUATION'] < 100000]
ren_budget['VALUATION'].plot(kind='hist', bins=20, title='Common Market: Renovations under $100K', xlabel='Valuation in USD ($)')

In [ ]:
ren = permits[permits['PERMIT_TYPE'] == 'Bldg-Alter/Repair'].copy()

# Create new column for month
ren['Month'] = ren['ISSUE_DATE'].dt.month_name()

# Group by the month and create a list of months
seasonal = ren.groupby('Month')['PCIS_PERMIT_NUM'].count()
order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Make order as the index before graphing the bar chart
seasonal = seasonal.reindex(order)
seasonal.plot(kind='bar', title='Seasonal Permit Volume')